In [17]:

print(49*48)
print(2.352/2.4)
print(int('0xBF68',0))

2352
0.98
49000


In [1]:
# Toets basiese skryf na die seriepoort
import serial
ser = serial.Serial('/dev/ttyUSB0')  # open serial port
print(ser.name)         # check which port was really used
#ser.write(b'hello')     # write a string
ser.close()             # close port

/dev/ttyUSB0


In [2]:
# Probeer 1200 baud en 115200
with serial.Serial('/dev/ttyUSB0', 1200, timeout=1) as ser:
    x = ser.read()          # read one byte
    s = ser.read(10)        # read up to ten bytes (timeout)
    #line = ser.readline()   # read a '\n' terminated line
    
print(s)

b''


In [4]:
from binascii import unhexlify
string='0281E1B1'
print(unhexlify(string))

b'\x02\x81\xe1\xb1'


In [30]:
from binascii import unhexlify

# Opdrag vanaf paragraaf 5 in seriepoort handleiding
bytestosend = '7E3230303134363432453030323031464433350D'

# Probeer 1200 baud en 115200
with serial.Serial('/dev/ttyUSB0', 1200, timeout=5.0) as ser:
    x = ser.write(unhexlify(bytestosend)) # Stuur opdrag na die battery
    
    uitstring = ser.read(1000)

# Druk die greepstring (bytestring)
print('Greepstring')
print(uitstring)

# Skakel die greepstring om na heksadesimale string
print('Heksadesimale string')
a = uitstring.hex()
print(a)    


Greepstring
b'~20014600F07A11010F0DE30DE30DD20DE30DE30DE40DE30DE20DE90DE70DDB0DE80DE70DE90DE7050BF50BC30BC30BC30BC30000D051FFFF04FFFF000F012110012110E20F\r'
Heksadesimale string
7e3230303134363030463037413131303130463044453330444533304444323044453330444533304445343044453330444532304445393044453730444442304445383044453730444539304445373035304246353042433330424333304243333042433330303030443035314646464630344646464630303046303132313130303132313130453230460d


In [17]:
# Deel die heks uitstring op in die dele wat die data bevat soos 
# die hoeveelheid energie oor in battery

print('Heksadesimale ASCII waarde:  aantal selle')
print(a[17:19])
print('Heksadesimale string:  aantal selle')
print(a[2:4])
print('Desimale string:  aantal selle')
print(bytearray.fromhex(a[2:4]).decode())

getalinheks = bytearray.fromhex("42463638").decode()
print(getalinheks)
print(int(getalinheks, 16))

Heksadesimale string:  aantal selle
04
Heksadesimale string:  aantal selle
32
2
BF68
49000
